In [ ]:
class KeyFrameDetector:
    def __init__(self):
        # Matrices para el mecanismo de atención (inicializadas aleatoriamente)
        self.feature_dim = 512
        self.W1 = np.random.randn(self.feature_dim, self.feature_dim) * 0.01
        self.W2 = np.random.randn(self.feature_dim, self.feature_dim) * 0.01
        self.W3 = np.random.randn(self.feature_dim, self.feature_dim) * 0.01
        
        # Capas adicionales para el cálculo de importancia
        self.U = np.random.randn(self.feature_dim, self.feature_dim) * 0.01
        self.L1_weights = np.random.randn(256, self.feature_dim) * 0.01
        self.L2_weights = np.random.randn(1, 256) * 0.01
    
    def extract_features_from_landmarks(self, landmarks_points):
        """Extrae un vector de características simplificado a partir de landmarks"""
        if landmarks_points is None:
            return None
            
        # Aplanar landmarks a un vector
        features = landmarks_points.reshape(-1)
        
        # Para una implementación real, aquí se usaría un modelo como ResNet
        # Simulamos un vector de dimensión feature_dim
        if len(features) > self.feature_dim:
            features = features[:self.feature_dim]
        elif len(features) < self.feature_dim:
            padding = np.zeros(self.feature_dim - len(features))
            features = np.concatenate([features, padding])
            
        return features
    
    def compute_importance_score(self, feature_sequence):
        """Implementa las fórmulas 1-6 del paper para calcular la importancia de frames"""
        K = len(feature_sequence)
        importance_scores = []
        
        for k in range(K):
            # Fórmula 1: Cálculo de correlación entre features
            alpha_k = np.zeros(K)
            for i in range(K):
                # αk,i = (W1pi)T(W2pk)
                alpha_k[i] = np.dot(np.dot(self.W1, feature_sequence[i]), 
                                   np.dot(self.W2, feature_sequence[k]))
            
            # Fórmula 2: Normalización con softmax
            # α˜k = Softmax(αk)
            alpha_k_normalized = softmax(alpha_k)
            
            # Fórmula 4: Ponderación de características
            # bk = Σ(i=1 to K) α˜k,i(W3pk)
            b_k = np.zeros_like(feature_sequence[k])
            for i in range(K):
                b_k += alpha_k_normalized[i] * np.dot(self.W3, feature_sequence[k])
            
            # Fórmula 5: Transformación y residual
            # gk = Norm(Dropout(Ubk + pk))
            # Simplificado sin dropout y normalización
            g_k = np.dot(self.U, b_k) + feature_sequence[k]
            
            # Fórmula 6: Capas finales para calcular la puntuación
            # zk = L2(L1(gk))
            l1_output = np.maximum(0, np.dot(self.L1_weights, g_k))  # ReLU
            score = 1 / (1 + np.exp(-np.dot(self.L2_weights, l1_output)[0]))  # Sigmoid
            
            importance_scores.append(score)
        
        return importance_scores
    
    def select_key_frames(self, frames_buffer, landmarks_buffer, movement_values):

    # Filtrar landmarks válidos y sus índices correspondientes
        valid_landmarks = [landmarks for landmarks in landmarks_buffer if landmarks is not None]
        valid_indices = [i for i, landmarks in enumerate(landmarks_buffer) if landmarks is not None]
    
        if len(valid_landmarks) < 3:  # Necesitamos al menos 3 puntos para calcular aceleración
            return []
    
    # 1. Calcular puntos de movimiento para un punto de referencia (p.ej. nariz o centro de la cara)
    # Usamos el punto central como representativo del movimiento facial
        punto_central = []
        for landmarks in valid_landmarks:
        # Calcular punto central promediando todos los landmarks
            if len(landmarks) > 0:
                puntos = landmarks.reshape(-1, 2)
                punto_central.append(np.mean(puntos, axis=0))
    
    # 2. Calcular distancias entre puntos consecutivos
        distancias = []
        for i in range(len(punto_central) - 1):
            dist = np.linalg.norm(punto_central[i+1] - punto_central[i])
            distancias.append(dist)
    
        if not distancias:
            return []
    
    # 3. Calcular velocidades acumulativas y velocidad instantánea
        dist_acum = np.cumsum(distancias)
        velocidades = []
        for i in range(len(dist_acum)):
            velocidades.append(dist_acum[i] / (i + 1))
    
    # 4. Calcular aceleraciones (cambio en velocidad)
        aceleraciones = []
        for i in range(len(velocidades) - 1):
            aceleraciones.append(velocidades[i+1] - velocidades[i])
    
        if not aceleraciones:
            return []
    
    # 5. Dividir la trayectoria en 3 segmentos y calcular desviación estándar para cada uno
        num_segmentos = min(3, len(aceleraciones))
        segmentos = []
        longitud_segmento = len(aceleraciones) // num_segmentos
    
        for i in range(num_segmentos):
            inicio = i * longitud_segmento
            fin = inicio + longitud_segmento if i < num_segmentos - 1 else len(aceleraciones)
            segmento = aceleraciones[inicio:fin]
            desviacion = np.std(segmento)
            segmentos.append((inicio, fin, desviacion))
    
    # 6. Seleccionar segmento con menor desviación estándar (movimiento más estable)
        segmento_estable = min(segmentos, key=lambda x: x[2])
        inicio_estable, fin_estable, _ = segmento_estable
    
    # 7. Seleccionar frames del segmento estable
    # Ajustar índices para corresponder a los frames originales
        inicio_frames = valid_indices[inicio_estable] if inicio_estable < len(valid_indices) else 0
        fin_frames = valid_indices[min(fin_estable, len(valid_indices)-1)]
    
    # Asegurarse de que el rango de frames es válido
        inicio_frames = max(0, inicio_frames)
        fin_frames = min(len(frames_buffer) - 1, fin_frames)
    
    # 8. Calcular nitidez para cada frame en el segmento estable
        frames_nitidez = []
        for i in range(inicio_frames, fin_frames + 1):
            if i >= len(frames_buffer):
                continue
            
            frame = frames_buffer[i]
            if frame is None:
                continue
            
        # Convertir a escala de grises para calcular nitidez
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Calcular nitidez usando el operador laplaciano
            nitidez = cv2.Laplacian(gray, cv2.CV_64F).var()
            frames_nitidez.append((i, nitidez))
    
    # 9. Seleccionar los frames más nítidos
        if not frames_nitidez:
            return []
        
    # Ordenar por nitidez y seleccionar los mejores
        frames_nitidez.sort(key=lambda x: x[1], reverse=True)
        num_frames_seleccionar = min(3, len(frames_nitidez))
        indices_seleccionados = [idx for idx, _ in frames_nitidez[:num_frames_seleccionar]]
        indices_seleccionados.sort()  # Mantener orden temporal
    ##xxx
    # 10. Devolver los frames seleccionados
        key_frames = [frames_buffer[idx] for idx in indices_seleccionados if idx < len(frames_buffer)]
        return key_frames